In [6]:
# prep for tts segment analysis 
# want to isolate each awas cannister in the UT and get TOGA measurements within time span
# date created: 2/28/2020
# author: sofia chelpon 

In [7]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns

import datetime as dt 

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [8]:
#### ------------------- TIME SPAN 
# 6 minutes total, 3 mins of either side of awas collection 
m = 6
speed = 235.4 #m/s
dist_km = (speed*(m*60))/1000
dist_km

84.744

In [9]:
#### ------------------- LOAD IN 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df.pkl")

In [61]:
#### ------------------- ISOLATE AWAS SEGMENTS IN THE UT 
awas_df_ut = awas_df[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)]
awas_df_ut.reset_index()

,index,Time,Flight,Notes,Intrument,GGALT,GGLAT,GGLON,C2H6_Ethane,C2H2_Ethyne,...,C2Br2F4_Halon_2402,CH2BrCl_Bromochloromethane,CH2Br2_Methylene_bromide,CHBrCl2_Bromodichloromethane,CHBr2Cl_Dibromochloromethane,CHBr3_Bromoform,C2H5ONO2_Ethyl_nitrate,C3H7NO3_Isopropyl_nitrate,C4H9NO3_2_Butyl_nitrate,C4H9NO3_n_Butyl_nitrate
0,13,2014-01-22 01:43:55,RF05,"Convective outflow, 2 - North of Guam",AWAS,12925.989258,12.914162,149.354141,418.0,38.0,...,0.43,0.14,0.99,0.48,0.25,0.74,3.23,1.62,0.67,0.00
1,14,2014-01-22 01:55:10,RF05,"Convective outflow, 2 - North of Guam",AWAS,12923.607422,14.304468,149.771027,422.0,42.0,...,0.41,0.13,0.98,NaN,0.23,0.70,3.25,1.52,0.78,0.00
2,15,2014-01-22 02:06:25,RF05,"Convective outflow, 2 - North of Guam",AWAS,12921.866211,15.739713,150.000473,360.0,34.0,...,0.41,0.12,0.96,NaN,0.21,0.67,3.07,1.38,0.54,0.00
3,16,2014-01-22 02:17:40,RF05,"Convective outflow, 2 - North of Guam",AWAS,12916.369141,17.191528,150.000214,344.0,34.0,...,0.40,0.15,0.99,NaN,0.24,0.80,3.79,1.38,0.65,0.00
4,17,2014-01-22 02:28:55,RF05,"Convective outflow, 2 - North of Guam",AWAS,12914.645508,18.010277,149.249603,556.0,75.0,...,0.41,0.17,1.01,0.42,0.25,0.81,4.13,2.66,0.99,1.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,552,2014-02-25 03:17:56,RF14,"Equator crossing, SH TTL survey, and Manus ozo...",AWAS,13443.239258,34.183247,145.000443,80.0,6.0,...,0.30,NaN,NaN,NaN,0.00,0.12,1.10,0.32,0.65,0.07
202,553,2014-02-25 03:26:56,RF14,"Equator crossing, SH TTL survey, and Manus ozo...",AWAS,13523.428711,33.037079,145.000412,267.0,19.0,...,NaN,NaN,NaN,NaN,0.04,0.00,1.00,0.21,0.59,0.00
203,554,2014-02-25 03:35:56,RF14,"Equator crossing, SH TTL survey, and Manus ozo...",AWAS,13597.672852,31.888105,145.000229,261.0,17.0,...,0.39,NaN,NaN,NaN,0.00,0.00,0.83,0.20,0.55,0.00
204,555,2014-02-25 03:48:58,RF14,"Equator crossing, SH TTL survey, and Manus ozo...",AWAS,13871.884766,30.309204,144.999847,364.0,42.0,...,0.42,NaN,NaN,NaN,0.04,0.12,0.93,0.25,0.54,0.04


In [46]:
#### ------------------- GET TOGA SEGMENTS THAT CORRESPOND TO AWAS TIMES 
# make a new dataframe for toga with AWAS time, plus average of all tracers in the segment
# want one UT value per segment 
toga_segments = pd.DataFrame()
for col in toga_df.columns: 
    toga_segments[col] = ''
toga_segments = toga_segments.drop(columns = 'Notes')

# grab everything TOGA +/- 3 minutes from that awas sample 
for idx in awas_df_ut.iterrows():
    t = idx[1].Time
    # ----- get time of awas sample, +/- 3 minutes
    m = 3
    s = 60*m
    ts = t - datetime.timedelta(0,s)
    te = t + datetime.timedelta(0,s) 
    # ----- get toga segment that corresponds to that time 
    toga_seg = toga_df[(toga_df.Time > ts) & (toga_df.Time < te)]
    toga_seg_sum = toga_seg.mean()
    # ----- if there are toga meas in that time, append 
    if (len(toga_seg) > 0):
        # ----- fill in what was lost from mean 
        time = pd.Series({'Time': t})
        flight = pd.Series({'Flight': toga_seg['Flight'].iloc[0]})
        inst = pd.Series({'Intrument': 'TOGA'})
        toga_seg_toappend = pd.DataFrame(pd.concat([time, flight, inst, toga_seg_sum]))
        toga_seg_toappend = toga_seg_toappend.transpose()
        # add to dataframe! 
        toga_segments = toga_segments.append(toga_seg_toappend, ignore_index=True)
    else:
        toga_seg_filled = toga_seg
        toga_seg_filled = toga_seg_filled.drop(columns = 'Notes')
        # fill ins, qualitative 
        time = pd.Series({'Time': t})
        flight = idx[1].Flight
        ggalt = idx[1].GGALT
        gglat = idx[1].GGLAT
        gglon = idx[1].GGLON
        inst = pd.Series({'Intrument': 'TOGA'})
        # tracers make nan 
        toga_seg_filled = toga_seg_filled.append(pd.Series(), ignore_index=True)
        toga_seg_filled['Time'] = t
        toga_seg_filled['Flight'] = flight[0]
        toga_seg_filled['Intrument'] = inst[0]
        toga_seg_filled['GGALT'] = ggalt
        toga_seg_filled['GGLAT'] = gglat
        toga_seg_filled['GGLON'] = gglon
        # add to dataframe
        toga_segments = toga_segments.append(toga_seg_filled, ignore_index=True)


toga_segments[toga_segments['Flight'] == 'RF06']

,Time,Flight,Intrument,GGALT,GGLAT,GGLON,Acetaldehyde,n_Butane,i_Butane,Benzene,C3H8,Methanol,Bromoform,Tetrachloroethylene,Dibromomethane,Dichloromethane,Chloroform,Carbon_Tetrachloride


In [12]:
#### ------------------- pickle and save 
awas_segments = awas_df_ut
awas_segments.to_pickle("./awas_segments.pkl")
toga_segments.to_pickle("./toga_segments.pkl")